In [108]:
# import các thư viện cần thiết
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [109]:
# Định Nghĩa lớp Node một nút trong cây quyết định
class Node:
  def __init__(self,Data, Label, right, left, infor_split):
    self.Data = Data # Định nghĩa dữ liệu tại node đó.
    self.Label = Label# Định nghĩa các nhãn tại node đó
    self.right = right # Định nghĩa node con bên phải
    self.left = left# Định nghĩa node con bên trái
    self.infor_split = infor_split # Định điều kiện rẽ nhánh tại node đó

Phương thức np.unique(): Trả về mảng đơn nhất các phần tử có trong mảng được chuyền vào
*   Thuộc tính return_counts = True => Đếm số lượng các phần tử => trả về mảng số lượng của các phần tử


In [110]:
# Định nghĩa mô hình phân lớp DecisionTreeClassifier
class DecisionTreeClassifier:
  def __init__(self,metrics = 'gini',max_depth = None, min_samples_leaf = None):
    # Định nghĩa tham số độ đo để xây dựng cây( GINI impurity: Độ không thuần khiết của DL, Entropy- Information Gain: Độ Ngạc nhiên TB - Thông tin bổ sung)
    self.metrics = metrics
    self.max_depth = max_depth
    self.min_samples_leaf = min_samples_leaf

  #------------------------------------ Định nghĩa hai độ đo Entropy: Độ ngạc nhiên TB của dữ liệu, GINI impurity: Độ không thuần khiết của dữ liệu ----------------------

  # Định nghĩa cách tính độ đo GINI impurity
  def compute_gini_impurity(self,Label):
    return 1 - np.sum((np.unique(Label,return_counts=True)[1]/Label.size)**2)

  # Định nghĩa cách tính độ đo Entropy: Độ ngạc nhiên trung bình (surprise)
  def compute_Entropy(self,Label):
    probability = np.unique(Label,return_counts=True)[1]/Label.size
    return -np.sum(probability*np.log2(probability))


  #--------------------------------------- Định nghĩa tính Total_Gini_Impurity và informationGain ---------------------------------------------
  def compute_InformationGain(self, Label_parent, Label_left, Label_right):
    Information_Gain = self.compute_Entropy(Label_parent) - ((Label_right.size/Label_parent.size)* self.compute_Entropy(Label_right) + (Label_left.size/Label_parent.size)* self.compute_Entropy(Label_left))
    return Information_Gain

  def compute_total_GINI(self, Label_parent, Label_left, Label_right):
    total_GINI = (Label_right.size/Label_parent.size)* self.compute_gini_impurity(Label_right) + (Label_left.size/Label_parent.size)* self.compute_gini_impurity(Label_left)
    return total_GINI


  # --------------------------------------Hàm tách dữ liệu theo thuộc tính và ngưỡng để tách dữ liệu -------------------------------------------
  def data_split(self,Data, Label, index_columns, threshold):
    X_right = Data[Data[:,index_columns] >= threshold]
    y_right = Label[Data[:,index_columns] >= threshold]
    X_left = Data[Data[:,index_columns] < threshold]
    y_left = Label[Data[:,index_columns] < threshold]
    return X_right, y_right, X_left, y_left

  # Định Hàm xây dựng cây quyết định
  def build_tree(self, Data, Label, func_metrics,depth = 0):
    if self.min_samples_leaf is not None and Data.shape[0] == self.min_samples_leaf:
      unique_Label, counts_label = np.unique(Label, return_counts=True)
      return Node(Data, unique_Label[counts_label.argmax()], None, None, None)

    if Label.size == 0 :
      return Node(None, None, None, None, None)

    # Nếu một nút chỉ tồn tại duy nhất nhất một loại nhãn <=> gini_impurity: độ không thuần khiết của dữ liệu  = 0 hoặc entropy: Độ ngạc nhiên trung bình của dữ liệu = 0
    # Hoặc  độ sâu bằng độ sâu quy định trước
    if np.unique(Label).size == 1 or depth == self.max_depth:
      unique_Label, counts_label = np.unique(Label, return_counts=True)
      return Node(Data, unique_Label[counts_label.argmax()], None, None, None)


    metric_and_threshold_split_of_each_column = []
    for index_columns in range(Data.shape[1]):
      # Lấy ra các giá trị đơn nhất của thuộc tính
      values_unique = np.unique(Data[:,index_columns])


      # Tham số là 1 tuple chứa giá trị của độ đo và ngưỡng tách
      metric_and_threshold = []

      if values_unique.size == 1:
        continue
      # dùng một cửa sổ trượt kernel 2x2 tính giá trị để tách giữ liệu ra
      for threshold in (values_unique[1:]+values_unique[:-1])/2:
        X_right, y_right, X_left, y_left = self.data_split(Data, Label, index_columns, threshold)
        if y_right.size or y_left.size == 0:
          continue
        metric_and_threshold.append((func_metrics(Label,y_left,y_right),threshold))
      if not metric_and_threshold:
        continue
      if self.metrics == 'gini':
        metric_and_threshold_split_of_each_column.append(min(metric_and_threshold,key=lambda x : x[0]))
      if self.metrics == 'entropy':
        metric_and_threshold_split_of_each_column.append(max(metric_and_threshold,key= lambda x : x[0]))
    # Nếu Không có ngưỡng nào tách dữ liệu hợp lệ sẽ trả về nhãn chiếm đa số của dữ liệu
    if not metric_and_threshold_split_of_each_column:
      unique_Label, counts_label = np.unique(Label, return_counts=True)
      return Node(Data, unique_Label[counts_label.argmax()], None, None, None)
    best_index_feature = metric_and_threshold_split_of_each_column.index(min(metric_and_threshold_split_of_each_column,key=lambda x : x[0])) if self.metrics == 'gini' else metric_and_threshold_split_of_each_column.index(max(metric_and_threshold_split_of_each_column,key=lambda x : x[0]))
    best_threshold_split = metric_and_threshold_split_of_each_column[best_index_feature][1]

    '''
    Tham số infor_split là 1 tuple gồm hai tham số
    Tham số 1: chỉ số của thuộc tính tốt nhất cho từng nút để tách dữ liệu.
    Tham số 2: Giá trị của thuộc tính tốt nhất để tách dữ liệu đó.
    '''
    infor_split = (best_index_feature,best_threshold_split)
    X_right, y_right, X_left, y_left = self.data_split(Data, Label, infor_split[0], infor_split[1])
    return Node(Data, Label,self.build_tree(X_right,y_right,func_metrics,depth=depth + 1),self.build_tree(X_left,y_left,func_metrics,depth = depth+1),infor_split)


  # ------------------------------------------------------------- Hàm huấn luyện mô hình ---------------------------------------------------------------
  def fit(self,X_train,y_train):
    self.X_train = X_train
    self.y_train = y_train
    if self.metrics == 'gini':
      self.root = self.build_tree(X_train,y_train,self.compute_total_GINI)
    if self.metrics == 'entropy':
      self.root = self.build_tree(X_train,y_train,self.compute_InformationGain)

  # --------------------------------------------------------------- Hàm dự đoán ------------------------------------------------------------------------
  def predict(self,X_test):
    self.X_test = X_test
    Node = self.root
    while Node.right is not None and Node.left is not None:
      if X_test[Node.infor_split[0]] >= Node.infor_split[1]:
        Node = Node.right
      else:
        Node = Node.left
    return Node.Label

In [111]:
data = {
    'age': ['youth', 'youth', 'middel_age', 'senior', 'senior', 'senior', 'middel_age', 'youth', 'youth', 'senior', 'youth', 'middel_age', 'middel_age', 'senior'],
    'income': ['high', 'high', 'high', 'medium', 'low', 'low', 'low', 'medium', 'low', 'medium', 'medium', 'medium', 'high', 'medium'],
    'student': ['no', 'no', 'no', 'no', 'yes', 'yes', 'yes', 'no', 'yes', 'yes', 'yes', 'no', 'yes', 'no'],
    'credit_rating': ['fair', 'excellent', 'fair', 'fair', 'fair', 'excellent', 'excellent', 'fair', 'fair', 'fair', 'excellent', 'excellent', 'fair', 'excellent'],
    'Default': ['no', 'no', 'yes', 'yes', 'yes', 'no', 'yes', 'no', 'yes', 'yes','yes', 'yes', 'yes', 'no']
}

In [112]:
df = pd.DataFrame(data,columns = data.keys())
df

,age,income,student,credit_rating,Default
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middel_age,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middel_age,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


In [113]:
Le = LabelEncoder()

In [114]:
df = df.apply(Le.fit_transform)
df

,age,income,student,credit_rating,Default
0,2,0,0,1,0
1,2,0,0,0,0
2,0,0,0,1,1
3,1,2,0,1,1
4,1,1,1,1,1
5,1,1,1,0,0
6,0,1,1,0,1
7,2,2,0,1,0
8,2,1,1,1,1
9,1,2,1,1,1


In [115]:
clf = DecisionTreeClassifier(metrics='entropy')

In [116]:
clf.fit(df.drop(columns=['Default']).values,df['Default'].values)